In [ ]:
import requests
import time
import json
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from kafka import KafkaClient
from kafka import SimpleProducer
from kafka import KafkaProducer
import sys

In [ ]:
YAHOO_URL = 'https://tw.news.yahoo.com/politics'
kafka = KafkaClient('localhost:9092')
producer = SimpleProducer(kafka, async=True)
topic_name = "test"

In [ ]:
def get_web_page(url):
    resp = requests.get(
        url=url,
        #cookies={'over18': '1'}
    )
    if resp.status_code != 200:
        print('Invalid url:', resp.url)
        return None
    else:
        return resp.text

In [ ]:
def get_articles(dom, date):
    #print(dom)
    soup = BeautifulSoup(dom, 'html5lib')

    divs = soup.find_all('div',class_='Py(14px) Pos(r)')
    #print(divs)
    for d in divs:
        # 取得文章連結及標題
        if d.find('a'):  # 有超連結，表示文章存在，未被刪除
            #href = d.find('a')['href']
            title = d.find('a').text
            sending = u'%s' % title
            print (sending)
            producer.send_messages('test', sending.encode('utf-8'))
            
    return articles

In [ ]:
if __name__ == '__main__':
    current_page = get_web_page(YAHOO_URL)
    if current_page:
        articles = []  # 全部的今日文章
        today = time.strftime("%m_%d_%H_%M")  # 今天日期
        
        options = Options()
        options.add_argument("--disable-notifications")
        options.add_argument("--start-maximized")

        driver = webdriver.Chrome(executable_path=r'C:\Users\Leo\Documents\project01\chromedriver',chrome_options=options) # chrome瀏覽器
        time.sleep(3)
        driver.get('https://tw.news.yahoo.com/politics')
        time.sleep(15)
        for i in range(8):  # 進行n次
            #driver.execute_script(js)    
            driver.execute_script('window.scrollTo(0,30000)')  # 重複往下捲動
            time.sleep(1)  # 每次執行打瞌睡一秒 
            
        pageSource = driver.page_source    
        HandsomeYuLi = get_articles(pageSource, today)  # 目前頁面的今日文章
             
driver.close()